# [Advent of Code 2022 Day ?]()

?

## Initial setup

In [ ]:
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2

## Input Parsing

In [ ]:
def parse_input(filename: str) -> Context:

    gen = yield_line(filename)

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for line in gen:
        if line == "noop":
            input_lines.append(("noop", None))
            continue
        assert line.startswith("addx")
        input_lines.append(("addx", int(line.removeprefix("addx "))))

    return ctx

## Sandbox

In [ ]:
%%ipytest --doctest-modules

def transform(x_value: int, incr: int) -> int:
    """
    >>> transform(1, 3)
    4
    """
    return x_value + incr

def execute(instructions) -> list[int]:
    """
    >>> execute([("noop", None), ("addx", 3), ("addx", -5)])
    [0, 1, 1, 1, 4, 4]
    >>> execute([("addx", 15), ("addx", 11)])
    [0, 1, 1, 16, 16]
    """

    ans = [0]
    work = []
    instructions.reverse()
    x = 1

    while work or instructions:
        curr_cycle = len(ans)
        if work:
            new_val = x + work.pop()
            # print(f"cycle={curr_cycle} changed x from {x} to {new_val}")
            ans.append(x)
            x = new_val
        else:
            if instructions:
                word, amt = instructions.pop()
                if word == "noop":
                    # print(f"cycle={curr_cycle} has no work, received noop, so more bloking")
                    pass
                elif word == "addx":
                    # print(f"cycle={curr_cycle} received work while having no work, going to pop this next cycle")
                    work.append(amt)
                else:
                    raise NotImplementedError(f"{word}")
            else:
                raise # this is actually impossible btw
            ans.append(x)

    return ans

def determine_position(curr_cycle: int) -> tuple[int, int]:
    """
    >>> determine_position(1)
    (0, 0)
    >>> determine_position(40)
    (0, 39)
    >>> determine_position(41)
    (1, 0)
    >>> determine_position(80)
    (1, 39)
    >>> determine_position(201)
    (5, 0)
    >>> determine_position(240)
    (5, 39)
    """
    assert curr_cycle != 0, f"Cycle zero doesn't exist moron"
    ans = divmod(curr_cycle - 1, 40)
    return ans[0], ans[1]

def get_sprite_pixels(pos: int) -> list[int]:
    # assert pos not in {0, 39}, f"{pos=} would create a sprite that is cut off the screen"
    # assert 0 <= pos <= 39, f"{pos=} out of range for current line"
    if not (0 <= pos <= 39):
        return []
    if pos == 0:
        return [0, 1]
    if pos == 39:
        return [38, 39]
    return [pos - 1, pos, pos + 1]

def draw(vals: list[int]) -> Any:
    screen = []
    for _ in range(6):
        screen.append(["x"] * 40)

    for cycle in range(1, len(vals)):
        pixels = get_sprite_pixels(vals[cycle])
        x, y = determine_position(cycle)
        if y in pixels:
            screen[x][y] = "#"
        else:
            screen[x][y] = "."

    for row in screen:
        print("".join(row))


def process(part: int, context: Context) -> int:
    s = execute(context.input)
    if part == 1:
        for i in range(len(s)):
            print(f"{i} -> {s[i]}")
        return 20 * s[20] + 60 * s[60] + 100 * s[100] + 140 * s[140] + 180 * s[180] + 220 * s[220]
    if part == 2:
        draw(s)
        return PART_TWO_SENTINEL
    else:
        raise Exception(f"Invalid part: {part}")

## Part 1
Lorem ipsum

In [ ]:
%%ipytest

def test_stuff():
    strengths = execute(parse_input("example2").input)
    assertions = (
        (20, 21),
        (60, 19),
        (100, 18),
        (140, 21),
        (180, 16),
        (220, 18),
    )
    for cycle, expect in assertions:
        if strengths[cycle] != expect:
            raise ValueError(f"For the example, expected cycle {cycle} to be {expect} but was actually {strengths[cycle]}")

def part_one(ctx: Context) -> int:
    return process(1, ctx)

## Part 1 Testing

In [ ]:
PART_ONE_CASES: dict[str, str | int] = {
    "example2": 13140,
    "input": 12740,  # not 14100
}
PART_ONE_OUTPUTS: dict[str, str | int] = dict()

In [ ]:
%%ipytest --doctest-modules
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES.items())
def test_part_one(test_file_name, test_expected_output):
    test_actual_output = part_one(parse_input(test_file_name))
    PART_ONE_OUTPUTS[test_file_name] = test_actual_output
    assert test_actual_output == test_expected_output

In [ ]:
for file_name, actual_output in PART_ONE_OUTPUTS.items():
    expected = PART_ONE_CASES[file_name]
    actual = actual_output
    outcome = ("SENTINEL" if actual == expected else "CANDIDATE") if expected == PART_ONE_SENTINEL else ("CORRECT" if expected == actual else "INCORRECT")
    print(f"{file_name:>10}      expected: {expected:>10}      actual: {actual:>10}      outcome: {outcome}")

## Part 2
Lorem ipsum

In [ ]:
def part_two(ctx: Context) -> int:
    return process(2, ctx)

## Part 2 Testing

In [ ]:
PART_TWO_CASES: dict[str, str | int] = {
    # ##..##..##..##..##..##..##..##..##..##..
    # ###...###...###...###...###...###...###.
    # ####....####....####....####....####....
    # #####.....#####.....#####.....#####.....
    # ######......######......######......####
    # #######.......#######.......#######.....
    "example2": PART_TWO_SENTINEL,  # Nonsense
    # ####..###..###...##..###...##...##..####
    # #...#.#..#.#..#.#..#.#..#.#..#.#..#.#...
    # #...#.###..#..#.#..#.#..#.#..#.#....###.
    # ####..#..#.###..####.###..####.#.##.#...
    # #..#..#..#.#....#..#.#.#..#..#.#..#.#...
    # #...#.###..#....#..#.#..#.#..#..###.#...
    "input": PART_TWO_SENTINEL,  # RBPARAGF
}
PART_TWO_OUTPUTS: dict[str, str | int] = dict()

In [ ]:
%%ipytest --doctest-modules -xrP
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES.items())
def test_part_two(test_file_name, test_expected_output):
    test_actual_output = part_two(parse_input(test_file_name))
    PART_TWO_OUTPUTS[test_file_name] = test_actual_output
    assert test_actual_output == test_expected_output

In [ ]:
for file_name, actual_output in PART_TWO_OUTPUTS.items():
    expected = PART_TWO_CASES[file_name]
    actual = actual_output
    outcome = ("SENTINEL" if actual == expected else "CANDIDATE") if expected == PART_TWO_SENTINEL else ("CORRECT" if expected == actual else "INCORRECT")
    print(f"{file_name:>10}      expected: {expected:>10}      actual: {actual:>10}      outcome: {outcome}")